# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [18]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [19]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle


___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***@Giiovana_lemes***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [20]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @Giiovana_lemes

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [21]:
#Produto escolhido:
produto = 'kit kat'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [22]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [23]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

In [24]:
#se quiser abrir a pasta que está sendo usada
#! explorer .


___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [31]:
%matplotlib inline
from mpmath import mpf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
plt.style.use('ggplot')
import mpmath

dados_treinamento = pd.read_excel('kit kat.xlsx', "Treinamento", sep=',')

In [32]:
#para pegar os dados da aba Treinamento
dados_treinamento = pd.read_excel('kit kat.xlsx', sheetname="Treinamento")
dados_treinamento = dados_treinamento.loc[:,["Treinamento", "Classificação"]]

In [33]:
#para fazer value counts ta aba Treinamento
vc_treinamento = dados_treinamento.Classificação.value_counts(True)

In [34]:
#Probabilidade do tweet ser relevante
prob_tweet_relevante = vc_treinamento.loc["relevante"]
prob_tweet_relevante

0.45666666666666667

In [35]:
#Probabilidade do tweet ser irrelevante
prob_tweet_irrelevante = vc_treinamento.loc["irrelevante"]
prob_tweet_irrelevante

0.54333333333333333

In [36]:
#para deixar todos os tweets como se fossem um único tweet "gigante"
tweet_gigante_treinamento = np.sum(dados_treinamento.Treinamento + " ") 

tweet_gigante_treinamento_irrel = np.sum(dados_treinamento[dados_treinamento.Classificação=="irrelevante"].Treinamento + " ")
tweet_gigante_treinamento_rel = np.sum(dados_treinamento[dados_treinamento.Classificação=="relevante"].Treinamento + " ")

#tweet_gigante_teste = np.sum(dados_teste.Teste + " ") 

qtd_relevante_treinamento = len(dados_treinamento[dados_treinamento.Classificação=="relevante"])
qtd_irrelevante_treinamento = len(dados_treinamento[dados_treinamento.Classificação=="irrelevante"])
qtd_total_tweets = qtd_relevante_treinamento + qtd_irrelevante_treinamento


In [37]:
#TIRAR A PONTUAÇÃO
import string

exclude = set(string.punctuation)  #pontuation pega todas as pontuaçoes 
def SemPontuacao(tweet_gigante):
    tweet_gigante = tweet_gigante.split() #para cada palavra do tweet gigante virar um tweet de palavra única
    lista_cada_palavra = []
    for s in tweet_gigante:
        for i in s:
            if i in exclude:
                s = s.replace(i,'') 
        lista_cada_palavra.append(s) #volta a ser um tweet gigante
    return lista_cada_palavra   
    
gigante_irrelevante = SemPontuacao(tweet_gigante_treinamento_irrel)
gigante_relevante = SemPontuacao(tweet_gigante_treinamento_rel)

gigante = SemPontuacao(tweet_gigante_treinamento)




In [65]:
#gigante

In [66]:
#para o tweet gigante virar uma Series (tabela)
palavras_irrelevante = pd.Series(gigante_irrelevante)
palavras_relevante = pd.Series(gigante_relevante)

#todas as palavras que aparecem relevantes e irrelevantes
palavras_totais = pd.Series(gigante)


In [67]:
#cria um data frame com as probabilidades
prob_total = pd.DataFrame(dados_treinamento.Classificação.value_counts(True)) 
prob_tweet_irrelevante = prob_total.Classificação[0] #prob total irrelevantes
prob_tweet_relevante = prob_total.Classificação[1] #prob total relevante

prob_total

,Classificação
irrelevante,0.543333
relevante,0.456667


In [68]:
qtd_irrelevante = len(dados_treinamento[dados_treinamento.Classificação=="irrelevante"]) #Quantidade de tweets irrelevantes
qtd_relevante = len(dados_treinamento[dados_treinamento.Classificação=="relevante"]) #Quantidade de tweets relevantes
qtd_irrelevante

163

In [69]:
#para saber quantas palavras diferentes existem entre relevantes e irrelevantes no TREINAMENTO
palavras_diferentes = len(palavras_totais.value_counts())

palavras_diferentes_relevante = len(palavras_relevante.value_counts())
palavras_diferentes_irrelevante = len(palavras_irrelevante.value_counts())

PP_relevante  = palavras_relevante.value_counts() #propabilidade palavra ser relevante
PP_irrelevante = palavras_irrelevante.value_counts() #propabilidade palavra ser irrelevante
palavras_dif = len(palavras_totais.value_counts()) #quantas palavras diferentes existem

#cada palavra e quantas vezes essa paravra apareceu
palavras_diferentes_relevante_qtd = (palavras_relevante.value_counts())
palavras_diferentes_irrelevante_qtd = (palavras_irrelevante.value_counts())

In [70]:
palavras_diferentes_irrelevante_qtd["kat"]

157

In [44]:
#para mostrar a palavra e quantas vezes ela aparece
for palavra in palavras_diferentes_relevante_qtd.index:
    print("palavra ", palavra, " aparece ", palavras_diferentes_relevante_qtd[palavra], " vezes")

palavra  kit  aparece  139  vezes
palavra  kat  aparece  137  vezes
palavra  de  aparece  61  vezes
palavra  um  aparece  41  vezes
palavra  que  aparece  29  vezes
palavra  é  aparece  27  vezes
palavra  e  aparece  26  vezes
palavra  eu  aparece  22  vezes
palavra  meu  aparece  19  vezes
palavra  chocolate  aparece  17  vezes
palavra  minha  aparece  16  vezes
palavra  com  aparece  16  vezes
palavra  branco  aparece  16  vezes
palavra  rt  aparece  15  vezes
palavra  me  aparece  14  vezes
palavra  comer  aparece  14  vezes
palavra  do  aparece  13  vezes
palavra  uma  aparece  13  vezes
palavra  a  aparece  13  vezes
palavra  queria  aparece  12  vezes
palavra  pra  aparece  12  vezes
palavra  melhor  aparece  12  vezes
palavra  o  aparece  11  vezes
palavra  vontade  aparece  11  vezes
palavra  da  aparece  10  vezes
palavra  mãe  aparece  10  vezes
palavra  bom  aparece  10  vezes
palavra  q  aparece  10  vezes
palavra  por  aparece  9  vezes
palavra  muito  aparece  8  vezes
pa

palavra  ou  aparece  1  vezes
palavra  yes  aparece  1  vezes
palavra  têm  aparece  1  vezes
palavra  dela  aparece  1  vezes
palavra  acabei  aparece  1  vezes
palavra  mm  aparece  1  vezes
palavra  bastiaalexandre  aparece  1  vezes
palavra  gnt  aparece  1  vezes
palavra  motivo  aparece  1  vezes
palavra  deuss  aparece  1  vezes
palavra  khaylaanacleto  aparece  1  vezes
palavra  liciaturola  aparece  1  vezes
palavra  9  aparece  1  vezes
palavra  grt  aparece  1  vezes
palavra  aosksols  aparece  1  vezes
palavra  recheio  aparece  1  vezes
palavra  geladeira  aparece  1  vezes
palavra  10  aparece  1  vezes
palavra  varios  aparece  1  vezes
palavra  aaaaa❤  aparece  1  vezes
palavra  👏  aparece  1  vezes
palavra  150  aparece  1  vezes
palavra  stress  aparece  1  vezes
palavra  meaturasim  aparece  1  vezes
palavra  churrasco  aparece  1  vezes
palavra  estiver  aparece  1  vezes
palavra  fui  aparece  1  vezes
palavra  seu  aparece  1  vezes
palavra  sexo  aparece  1  vez

## Criando classificador para o treinamento 

### Calculando a probabilidade de cada palavra aparecer (em relevante em irrelevante)

In [71]:
#para calcular a propabilidade de cada palavra aparecer em Relevante
P_palavra_relevante = (PP_relevante+1)/(qtd_relevante + palavras_dif) #lista com probabilidade das palavras quando tw relevante

#para calcular a propabilidade de cada palavra aparecer em Irrelevante
P_palavra_irrelevante = (PP_irrelevante+1)/(qtd_irrelevante + palavras_dif) #lista com probabilidade das palavras quando tw neutro

P_palavra_relevante

kit              0.107116
kat              0.105585
de               0.047437
um               0.032135
que              0.022953
é                0.021423
e                0.020658
eu               0.017598
meu              0.015302
chocolate        0.013772
minha            0.013007
com              0.013007
branco           0.013007
rt               0.012242
me               0.011477
comer            0.011477
do               0.010712
uma              0.010712
a                0.010712
queria           0.009946
pra              0.009946
melhor           0.009946
o                0.009181
vontade          0.009181
da               0.008416
mãe              0.008416
bom              0.008416
q                0.008416
por              0.007651
muito            0.006886
                   ...   
ela              0.001530
yakisoba         0.001530
sou              0.001530
pessoas          0.001530
ee               0.001530
pouco            0.001530
acabou           0.001530
mpn         

# Analisando Teste

In [53]:
#para pegar os dados da aba Teste
dados_teste = pd.read_excel('kit kat.xlsx', "Teste", sep=',')

dados_teste = pd.read_excel('kit kat.xlsx', sheetname="Teste")
dados_teste = dados_teste.loc[:,["Teste", "Classificação"]]

In [54]:
#para fazer value counts ta aba teste
vc_teste = dados_teste.Classificação.value_counts(True)

In [55]:
#Probabilidade do tweet ser relevante
vc_teste.loc["relevante"]

0.36683417085427134

In [56]:
#Probabilidade do tweet ser irrelevante
vc_teste.loc["irrelevante"]

0.63316582914572861

In [57]:
lista_teste = dados_teste.Teste

In [72]:
#Tirando e pontuação e separando cada tweet em uma lista e depois, cada palavra desse tweet dentro dessa lista

exclude = set(string.punctuation) 

def SemPontuacao(tweet_gigante):
    lista_cada_palavra_do_tweet = []
    for g in range(len(tweet_gigante)):
        lista_tweets_splitados = tweet_gigante[g].split()
        lista_tweets = []

        for f in lista_tweets_splitados:
            for i in f:
                if i in exclude:
                    f = f.replace(i,'') 
                
            lista_tweets.append(f)
        lista_cada_palavra_do_tweet.append(lista_tweets)
                
    return lista_cada_palavra_do_tweet   


lista_palavras_tweet = SemPontuacao (lista_teste)

#cria lista sem pontuação
teste_limpo = SemPontuacao(lista_teste)

#####

def Prob_cada_tweet(listafrase, p_palavra, p_tweet, qtd_tweet):
    listaP = []
    for frase in listafrase:
        P_frase = mpmath.mpf(1.000)
        for p in frase:
            if p in p_palavra:
                P_frase = P_frase * p_palavra[p] * p_tweet
            else:
                P_frase = mpmath.mpf(1)/(qtd_tweet + palavras_diferentes) * P_frase * p_tweet
        listaP.append(P_frase)
    return listaP



##Cálculo da probabilidade do tweet ser irrelevante
teste_prob_irrelevante = Prob_cada_tweet(teste_limpo,P_palavra_irrelevante,prob_tweet_irrelevante,qtd_irrelevante)

#Cálculo da probabilidade do tweet ser relevante
teste_prob_relevante = Prob_cada_tweet(teste_limpo,P_palavra_relevante,prob_tweet_relevante,qtd_relevante)


In [81]:
#agora temos que comparar a lista de probabilidade feita no teste, e ver qual prevalece, se é neutro ou relevante
def Comparador(teste_prob_irrelevante,teste_prob_relevante):
    resultado = []
    for i in range(len(teste_prob_irrelevante)):
        if teste_prob_irrelevante[i] < teste_prob_relevante[i]:
            resultado.append('relevante')
        elif teste_prob_irrelevante[i] > teste_prob_relevante[i]:
            resultado.append('irrelevante')
    return resultado

#lista de respostas que esse codigo vai gerar
resultado_teste = Comparador(teste_prob_irrelevante, teste_prob_relevante)


In [82]:
resultado_teste

['irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'relevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'relevante',
 'irrelevante',
 'relevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'relevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'relevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'relevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'relevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'irrelevante',
 'relevante',
 'releva

In [83]:
#para criar uma tabela com os tweets, a classificação da base e a classificação feita pelo comparador
data = {'Tweets': teste_limpo, 'Classificação': dados_teste.Classificação, 'Respostas': resultado_teste}

tabelaT = pd.DataFrame(data, columns = ['Tweets','Classificação','Respostas'])
tabelaT

,Tweets,Classificação,Respostas
0,"[indo, p, big, só, p, comprar, kit, kat]",relevante,irrelevante
1,"[kkkkkk, pior, que, já, aquele, kit, kat, mara...",irrelevante,irrelevante
2,"[tô, viciado, em, comer, kit, kat]",relevante,irrelevante
3,"[rt, xxalany, , 1941, alany, 💋, tocou, aqui, e...",irrelevante,irrelevante
4,"[minha, mãe, quer, definitivamente, me, ver, c...",irrelevante,irrelevante
5,"[minha, mãe, fez, sorvete, de, kit, kat, eu, a...",relevante,relevante
6,"[159, 1456, só, faço, se, tú, comprar, um, kit...",irrelevante,irrelevante
7,"[gabeharmonizer, como, com, menos, frequência,...",irrelevante,irrelevante
8,"[cheguei, da, rua, e, meu, pai, disse, tem, ki...",relevante,irrelevante
9,"[masoqmauricioo, não, é, bolo, mano, é, uma, t...",irrelevante,irrelevante


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [86]:
#verificação feita com base dos critérios determinados acima 
verificacao = []
for i in range(len(resultado_teste)):
    if dados_teste.Classificação[i] == resultado_teste[i]:
        if resultado_teste[i] == 'relevante':
            verificacao.append('positivo verdadeiro')
        elif resultado_teste[i] == 'irrelevante':
            verificacao.append('negativo verdadeiro')
            
    elif dados_teste.Classificação[i] != resultado_teste[i]:
        if resultado_teste[i] == 'relevante':
            verificacao.append('positivo falso')
        elif resultado_teste[i] == 'irrelevante':
            verificacao.append('negativo falso')
    
    

In [87]:
verificacao_final = pd.Series(verificacao)
porcentagem = verificacao_final.value_counts(True)
porcentagem



negativo verdadeiro    0.615
negativo falso         0.290
positivo verdadeiro    0.080
positivo falso         0.015
dtype: float64

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


### Conclusão:

   Na verificação do classificador, os dados negativos verdadeiros (treinamento e teste iguais e irrelevantes) se destacaram, isso ocorre pois os tweets com conteúdo desprezível ou negativo sobre o chocolate Kit Kat eram mais perceptíveis, de forma a sempre terem palavras relacionadas ao compartilhamento de algum video no Youtube ou simplismente a demonstração clara de desgosto pelo produto como, "odeio", "não gosto" ou "ruim", termos esses que praticamente não aparecem em tweets considerados positivos para a marca. Assim, a compatibilidade quanto aos comentários do treinamento e teste pode ser explicada pela presença frequente das mesmas expressões. 

   Os outros resultados apresentados quanto ao negativo falso, positivo verdadeiro e positivo falso, podem ser explicados pela dificuldade na classificação dos tweets relevantes, pois da mesma maneira que muitas pessoas expressam o seu apreço pelo chocolate em si, muitas delas apenas comentam sobre receitas como, bolos, sorvetes, tortas, etc..., que contém o produto como ingrediente principal. Dessa maneira, alguns posts que falavam sobre o preparo dos doces foram classificados como irrelevantes, porém, a maioria dos tweets relevantes eram sobre usuários elogiando determinado doce feito com Kit Kat, o que não pode ser desconsiderado, uma vez que os clientes da marca podem consumir seu produto de diferentes formas, mas por conta desse inconveniente, alguns comentários positivos foram considerados negativos e vice-versa, devido a presença de palavras em comum.
   
   Outro problema que também pode ser levado em conta é o fato de que o classificador analisa a probabilidade das palavras aparecerem em determinado tweet, porém, alguns usuários utilizam do sarcasmo para expressar a sua simpatia pelo produto e isso só pode ser compreendido quando a classificação é feita por alguém que consiga compreender o significado de tal expressão, o que no caso não pode ser determinado pelo classificador através das probabilidades, gerando divergencia entre os resultados.
   
   O classificador, no momento, não pode ser considerado extremamente eficaz devido as explicações citadas anteriormente, mas pode-se dizer que aumentando o direcionamento dos tweets considerados relevantes, os resultados obtidos serão mais precisos e ajudarão a empresa na análise dos dados sobre a satisfação de seus clientes. Portanto, as divergências atuais podem ser consertadas com algumas pesquisas e categorizações minuciosas, porém, após tal evolução o classificador terá um excelente desempenho servindo como base para qualquer exploração. 